# MSCI 434 Term Project

## 1. Import Packages and Licenses

In [1]:
%pip install gurobipy
import gurobipy as gp
from gurobipy import GRB as GRB

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
e = gp.Env(empty = True)
e.setParam('WLSACCESSID', 'a1afce0b-a3ec-4cb8-bd34-adb6cf4ca7b4')
e.setParam('WLSSECRET', '8af544aa-7014-4a6d-8e19-ddef9cdcda34')
e.setParam('LICENSEID', 2397655)
e.start()

model = gp.Model(env = e)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2397655


GurobiError: Unauthorized access